# Projeto 1 - Ciência dos Dados

Nome: Danilo Ramos

Nome: Leonardo Veras

Nome: Thomas Ganut

Atenção: Serão permitidos grupos de até 4 pessoas, mas com uma rubrica mais exigente. 

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from IPython.display import display
import re

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\danil\Documents\GitHub\C dados


Carregando a base de dados com as mensagens dos seus arquivos:

In [3]:
arquivo_treino = 'dados_treino_QUARTETO_Danilo_Leonardo_Thomas.csv'
train = pd.read_csv(arquivo_treino)

train.head(5)

,Comment,Topic
0,If you pile up electrons in a hypothetical cha...,Physics
1,Is it the infamous kill-you-dead Brazilian wan...,Biology
2,What a dumb fucking question. No you moron the...,Biology
3,Yes! Humans in North America have really led ...,Biology
4,Take my award. What an underrated comment.,Chemistry


In [4]:
test = pd.read_csv('dados_teste_QUARTETO_Danilo_Leonardo_Thomas.csv')
test.head(5)

,Comment,Topic
0,Wait wouldn’t the wings rip off before the res...,Biology
1,It's pretty much the same idea behind people a...,Biology
2,"Yeah, good to know. We had a crazy infestation...",Biology
3,It’s a kiwi doggy,Biology
4,You're absolutely right about the flu not kill...,Biology


___
## Classificador automático


Faça aqui uma descrição do seu assunto e o contexto referente aos rótulos cujas mensagens (ou reviews) deverão ser classificadas.

Será desenvolvido um sistema automático, utilizando o Teorema de Bayes, para classificar textos científicos em física, química e biologia. Uma prova de conceito será implementada para treinar e testar o classificador, garantindo a separação clara dos textos nas respectivas áreas.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [5]:
def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[´"!-.:?;$'')(&€™/]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

In [6]:
list_train: list = []
for frase in train.Comment:
    frase = frase.split()
    for word in frase:
        list_train.append(word)
        
serie_train = pd.Series(list_train)
tabela = serie_train.value_counts()
stop_words = tabela.index[:15].tolist()
for i in range(len(stop_words)):
    stop_words[i] = stop_words[i].lower()
print(stop_words)

['the', 'to', 'a', 'of', 'and', 'is', 'in', 'i', 'you', 'that', 'it', 'for', 'be', 'are', 'with']


In [7]:
def cleanup_stopwords(frase):
    punctuation = '[´"!-.:?;$'')(&€™/]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', frase)
    text_subbed = text_subbed.lower()
    text_subbed = text_subbed.replace(("\n"),' ')
    for palavra in stop_words:
        text_subbed = text_subbed.replace((f' {palavra} '), ' ')

    return text_subbed

print(cleanup_stopwords('shes contrarian who wants tear down anything she sees as being hyped this particular case shes totally right do so honestly amount bullshit surrounding delayed choice quantum eraser makes clear case sabines particular style being warranted perhaps even necessary\n\nshe gets hate because there other instances where her contrarian nature runs totally contrary what experts say  what we have pretty good reason believe her hot takes often foster misunderstanding can sometimes see her acolytes calling bullshit on say string theory or dark matter when they clearly havent taken time understand why people believe these things also there accusations she intentionally divisive increase her popular appeal sell more books which probably partially true'))

shes contrarian who wants tear down anything she sees as being hyped this particular case shes totally right do so honestly amount bullshit surrounding delayed choice quantum eraser makes clear case sabines particular style being warranted perhaps even necessary  she gets hate because there other instances where her contrarian nature runs totally contrary what experts say  what we have pretty good reason believe her hot takes often foster misunderstanding can sometimes see her acolytes calling bullshit on say string theory or dark matter when they clearly havent taken time understand why people believe these things also there accusations she intentionally divisive increase her popular appeal sell more books which probably partially true


In [8]:
for i, frase in enumerate(train.Comment):
    train.Comment[i] = cleanup_stopwords(frase.lower())

list_train: list = []
for palavra in train.Comment:
    frase = palavra.split()
    for word in palavra:
        list_train.append(word)
        
serie_train = pd.Series(list_train)
#for i, palavra in enumerate(test.Comment):
#    test.Comment[i] = cleanup(frase.lower())


In [9]:
train_physics = train.loc[train.Topic == 'Physics']
train_biology = train.loc[train.Topic == 'Biology']
train_chemistry = train.loc[train.Topic == 'Chemistry']

In [10]:
list_chemistry: list = []
for frase in train_chemistry.Comment:
    frase = frase.split()
    for word in frase:
        list_chemistry.append(word)

serie_chemistry = pd.Series(list_chemistry)


list_physics: list = []
for frase in train_physics.Comment:
    frase = frase.split()
    for word in frase:
        list_physics.append(word)

serie_physics = pd.Series(list_physics)


list_biology: list = []
for frase in train_biology.Comment:
    frase = frase.split()
    for word in frase:
        list_biology.append(word)

serie_biology = pd.Series(list_biology)

In [19]:
P_C = len(serie_chemistry)/len(serie_train)
P_B = len(serie_biology)/len(serie_train)
P_P = len(serie_physics)/len(serie_train)
print(P_C, P_B, P_P)
print(P_C + P_B + P_P)


0.04618754369214049 0.058627729759854794 0.04863916789317367
0.15345444134516895


In [12]:
# #Treinamento
tabela_c_relativa = serie_chemistry.value_counts(True)
tabela_p_relativa = serie_physics.value_counts(True)
tabela_b_relativa = serie_biology.value_counts(True)
print(tabela_p_relativa['this'])

voc = tabela.count()
# Physics
tabela_P_raw = serie_physics.value_counts()
tabela_P_freq = tabela_P_raw.sum()
tabela_P_voc = tabela_P_raw.count()

# Chemistry
tabela_C_raw = serie_chemistry.value_counts()
tabela_C_freq = tabela_C_raw.sum()
tabela_C_voc = tabela_C_raw.count()

# Biology
tabela_B_raw = serie_biology.value_counts()
tabela_B_freq = tabela_B_raw.sum()
tabela_B_voc = tabela_B_raw.count()

frase_teste = 'My name is the Leo, and I play a lot of Minecraft!'

0.012086750292882247


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [13]:
def algoritmo_1(frase: str):
    probPalavraDadoP = 1
    probPalavraDadoC = 1
    probPalavraDadoB = 1
    
    for palavra in frase.split():
        if palavra in serie_biology:
            probPalavraDadoB *= tabela_b_relativa[palavra]
        else:
            probPalavraDadoB *= 1

        if palavra in serie_physics:
            probPalavraDadoP *= tabela_p_relativa[palavra]
        else:
            probPalavraDadoP *= 1

        if palavra in serie_chemistry:
            probPalavraDadoC *= tabela_c_relativa[palavra]
        else:
            probPalavraDadoC *= 1
        
    if probPalavraDadoP == 0:
            print(frase)


    prob_P_dado_frase = probPalavraDadoP * P_P
    prob_C_dado_frase = probPalavraDadoC * P_C
    prob_B_dado_frase = probPalavraDadoB * P_B

    # print(prob_P_dado_frase)
    # print(prob_C_dado_frase)
    # print(prob_B_dado_frase)
    # print(max(prob_P_dado_frase, prob_C_dado_frase, prob_B_dado_frase))
    # print('\n')


    if  prob_P_dado_frase > prob_C_dado_frase:
        if prob_P_dado_frase > prob_B_dado_frase:
            return 'Physics'
        else:
            return 'Biology'
    else:
        if prob_C_dado_frase > prob_B_dado_frase:
            return 'Chemistry'
        else:
            return 'Biology'
        


#frases_physics = test.loc[test.Topic == 'Physics']
#frases_chemistry = test.loc[test.Topic == 'Chemistry']
#frases_biology = test.loc[test.Topic == 'Biology']



In [14]:
def algoritmo_laplace(frase: str):
    alpha = 1 # Suavizador de Laplace
    probPalavraDadoP = 1
    probPalavraDadoC = 1
    probPalavraDadoB = 1

    for palavra in frase.split():
        if palavra in serie_biology:
            probPalavraDadoB *= (tabela_B_raw[palavra] + alpha) / (tabela_B_freq + alpha * voc)
        else:
            probPalavraDadoB *= alpha / (tabela_B_freq + alpha * voc)

        if palavra in serie_physics:
            probPalavraDadoP *= (tabela_P_raw[palavra] + alpha) / (tabela_P_freq + alpha * voc)
        else:
            probPalavraDadoP *= alpha / (tabela_P_freq + alpha * voc)

        if palavra in serie_chemistry:
            probPalavraDadoC *= (tabela_C_raw[palavra] + alpha) / (tabela_C_freq + alpha * voc)
        else:
            probPalavraDadoC *= alpha / (tabela_C_freq + alpha * voc)
             
    prob_P_dado_frase = probPalavraDadoP #* P_P
    prob_C_dado_frase = probPalavraDadoC #* P_C
    prob_B_dado_frase = probPalavraDadoB #* P_B

    print(prob_P_dado_frase)
    print(prob_C_dado_frase)
    print(prob_B_dado_frase)
    print('\n')


    if  prob_P_dado_frase > prob_C_dado_frase:
        if prob_P_dado_frase > prob_B_dado_frase:
            return 'Physics'
        else:
            return 'Biology'
    else:
        if prob_C_dado_frase > prob_B_dado_frase:
            return 'Chemistry'
        else:
            return 'Biology'

In [15]:
frase_teste = cleanup_stopwords('honestly amount bullshit surrounding delayed choice quantum eraser makes clear case sabines particular style being warranted perhaps even necessary\n\nshe gets hate because there other instances where her contrarian nature runs totally contrary what experts say  what we have pretty good reason believe her hot takes often foster misunderstanding can sometimes see her acolytes calling bullshit on say string theory or dark matter when they clearly havent taken time understand')
print(frase_teste)
print('Sem Laplace: \n', algoritmo_1(frase_teste), '\n')
# print('Com Laplace: \n', algoritmo_laplace(frase_teste))

honestly amount bullshit surrounding delayed choice quantum eraser makes clear case sabines particular style being warranted perhaps even necessary  she gets hate because there other instances where her contrarian nature runs totally contrary what experts say  what we have pretty good reason believe her hot takes often foster misunderstanding can sometimes see her acolytes calling bullshit on say string theory or dark matter when they clearly havent taken time understand
Sem Laplace: 
 Biology 



In [16]:
validacao = test
validacao['algo1_certa'] = 'NaN'
# validacao['algoL_certa'] = 'NaN'
validacao['algo1'] = 'NaN'
# validacao['algoL'] = 'NaAAAN'

for i, frase in enumerate(test.Comment):
    frase = cleanup(frase)
    validacao['algo1'][i] = algoritmo_1(frase)
    # validacao['algoL'][i] = algoritmo_laplace(frase)
  
    if validacao['algo1'][i] == validacao['Topic'][i]:
        validacao['algo1_certa'][i] = True
    else:
        validacao['algo1_certa'][i] = False

    # if validacao['algoL'][i] == validacao['Topic'][i]:
        # validacao['algoL_certa'][i] = True
    # else:
        # validacao['algoL_certa'][i] = False

validacao.head(15)

,Comment,Topic,algo1_certa,algo1
0,Wait wouldn’t the wings rip off before the res...,Biology,True,Biology
1,It's pretty much the same idea behind people a...,Biology,True,Biology
2,"Yeah, good to know. We had a crazy infestation...",Biology,True,Biology
3,It’s a kiwi doggy,Biology,True,Biology
4,You're absolutely right about the flu not kill...,Biology,True,Biology
5,As long as the don't start having fire spurts ...,Biology,True,Biology
6,Beneficial to peoples gardens but are very bad...,Biology,True,Biology
7,"No, Triatoma can transmit Chagas, assassin bug...",Biology,True,Biology
8,I don’t think it’s a hormone mediated response...,Biology,True,Biology
9,I was under the impression that the silicate b...,Chemistry,False,Biology


In [17]:
# certos_algo1 = validacao.algo1_certa.value_counts(True)
# certos_algoL = validacao.algoL_certa.value_counts(True)

# print('Acertos sem Laplace: ')
# print(f'{certos_algo1[True].round(2)*100}%')
# print('\nAcertos com Laplace:')
# print(f'{certos_algoL[True].round(2)*100}%')

In [18]:
# pd.crosstab(test.Topic, test.algo1, normalize=True)
# print(test.algo1.value_counts())
print('\n')
print(test.algo1.value_counts())



algo1
Biology    1800
Name: count, dtype: int64


___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações das mensagens entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Leia atentamente a rubrica colocada no enunciado do Projeto 1 (última página). <br>
Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nas mensagens, mas tendem a melhorar na classificação das mensagens. Ex: stemming, lemmatization, stopwords.
* CONSIDEROU arquivo com três categorias na classificação das variáveis (OBRIGATÓRIO PARA QUARTETOS, sem contar como item avançado)
* CONSTRUIU o cálculo das probabilidades corretamente utilizando bigramas E apresentou referência sobre o método utilizado.
* EXPLICOU porquê não pode usar novas mensagens classificadas pelo próprio classificador como amostra de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários diferentes, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* REFLETE criticamente sobre os resultados obtidos, identificando limitações do modelo e sugerindo possíveis melhorias ou diferentes abordagens com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa).
* DOCUMENTOU bem o código, com explicações claras para cada etapa do processo, incluindo o raciocínio por trás das decisões de modelagem e das transformações de dados.


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**